# HW8 (20')

<font size='4'>

For this assignment, it is a combination of jupyter notebook assignment and python scripts.

For Q1, please upload your outputs including codes and graphics to your own GitHub repository. <br> You will need to disclose your GitHub repository below.

For Q2, please submit this jupyter notebook as an HTML or PDF file.

First of all, print your name (First and Last) below.

In [1]:
print('Tiyamika Williams')

Tiyamika Williams


## 0. Import relevant packages

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf

## Q1. Convert your HW7 to python scripts. (10')

<font size='4'>

- Under your working directory, there should be a folder called `self_py_fun`.
- Create a new python file called `HW8Fun.py` and move previously defined functions `produce_trun_mean_cov()`, `plot_trunc_mean()`, and `plot_trunc_cov()` to that file. Make sure you import proper packages.
- Create another main file `HW8_main.py`.
- Import relevant packages, modules, and/or function.
- Copy the global variables and call your functions inside `HW8_main.py`.
- A major difference compared to HW7 is that you are asked to save those figures to your local working environment.
    - Create a new directory `K114` under your current working directory.
    - For mean functions, please save it as a `Mean.png` output using `plt.savefig()` function.
    - The changes should be made within `HW8Fun.py` rather than `HW8_main.py`.
    - For covariance matrices, please save them as `Covariance_Target.png`, `Covariance_Non-Target.png`, and `Covariance_All.png` outputs using the same function above.
    - To summarize, there should be **four** figures under `K114` folder.
- Upload your entire work to your GitHub repository via push button.

In [90]:
# Provide your GitHub repository link below in the Markdown chunk. Remember to make it public and make the link clickable.
# Do not include sensitive information in your GitHub repository.

In [12]:
def produce_trun_mean_cov(input_signal, input_type, E_val):
    r"""
    args:
    -----
        input_signal: 2d-array, (sample_size_len, feature_len)
        input_type: 1d-array, (sample_size_len,)
        E_val: integer, (number of electrodes)

    return:
    -----
        A list of 5 arrays including
            signal_tar_mean, (E_val, length_per_electrode)
            signal_ntar_mean, (E_val, length_per_electrode)
            signal_tar_cov, (E_val, length_per_electrode, length_per_electrode)
            signal_ntar_cov, (E_val, length_per_electrode, length_per_electrode)
            signal_all_cov, (E_val, length_per_electrode, length_per_electrode)

    note:
    -----
        descriptive mean and sample covariance statistics from real data
        In this case, E_val=16, length_per_electrode=25. 
        But you should pass them as arguments or calculate them inside the function.
    """

    length_per_electrode = input_signal.shape[1] // E_val

    # Separate by type
    signal_tar = input_signal[input_type == 1]
    signal_ntar = input_signal[input_type == -1]

    # Reshape into (n_trials, E_val, length_per_electrode)
    signal_tar = signal_tar.reshape(-1, E_val, length_per_electrode)
    signal_ntar = signal_ntar.reshape(-1, E_val, length_per_electrode)
    signal_all = input_signal.reshape(-1, E_val, length_per_electrode)

    # Compute means
    signal_tar_mean = np.mean(signal_tar, axis=0)
    signal_ntar_mean = np.mean(signal_ntar, axis=0)

    # Compute covariances
    signal_tar_cov = np.array([np.cov(signal_tar[:, e, :].T) for e in range(E_val)])
    signal_ntar_cov = np.array([np.cov(signal_ntar[:, e, :].T) for e in range(E_val)])
    signal_all_cov = np.array([np.cov(signal_all[:, e, :].T) for e in range(E_val)])

    # Print shape info
    print("signal_tar_mean shape:", signal_tar_mean.shape)
    print("signal_ntar_mean shape:", signal_ntar_mean.shape)
    print("signal_tar_cov shape:", signal_tar_cov.shape)
    print("signal_ntar_cov shape:", signal_ntar_cov.shape)
    print("signal_all_cov shape:", signal_all_cov.shape)
    return [signal_tar_mean, signal_ntar_mean, signal_tar_cov, signal_ntar_cov, signal_all_cov]

def plot_trunc_mean(
    eeg_tar_mean, eeg_ntar_mean, subject_name, time_index, E_val, electrode_name_ls,
    y_limit=np.array([-5, 8]), fig_size=(12, 12)
):
    r"""
    :param eeg_tar_mean:
    :param eeg_ntar_mean:
    :param subject_name:
    :param time_index:
    :param E_val:
    :param electrode_name_ls:
    :param y_limit: optional parameter, a list or an array of two numbers
    :param fig_size: optional parameter, a tuple of two numbers
    :return:
    """


    fig, axes = plt.subplots(4, 4, figsize=fig_size)
    axes = axes.flatten()

    for i in range(E_val):
        ax = axes[i]
        ax.plot(time_index, eeg_tar_mean[i], color='red', label='Target')
        ax.plot(time_index, eeg_ntar_mean[i], color='blue', label='Non-Target')
        ax.set_title(electrode_name_ls[i])
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Amplitude (μV)')
        ax.set_ylim(y_limit)
        if i == 0:
            ax.legend()

    for j in range(E_val, len(axes)):
        fig.delaxes(axes[j])

    fig.suptitle(f"{subject_name}: Target vs Non-Target Sample Means", fontsize=14)
    plt.tight_layout()
    plt.show()

def plot_trunc_cov(
          eeg_cov, cov_type, time_index, subject_name, E_val, electrode_name_ls, fig_size=(14,12)
):
    X, Y = np.meshgrid(time_index, time_index)

    vmin = np.min(eeg_cov)
    vmax = np.max(eeg_cov)

    fig, axes = plt.subplots(4, 4, figsize=fig_size, constrained_layout=True)
    axes = axes.flatten()

    for i in range(E_val):
        ax = axes[i]
        cf = ax.contourf(X, Y, eeg_cov[i], levels=20, cmap='viridis', vmin=vmin, vmax=vmax)
        ax.set_title(electrode_name_ls[i])
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Time (ms)')
        ax.invert_yaxis()

    cbar = fig.colorbar(cf, ax=axes, orientation='vertical', fraction=0.03, pad=0.02)
    cbar.set_label('Covariance (μV²)')
    fig.suptitle(f"{subject_name}: {cov_type} Covariance", fontsize=14)
    plt.show()
    pass

from self_py_fun.HW8Fun import (
    produce_trun_mean_cov,
    plot_trunc_mean,
    plot_trunc_cov
)

ModuleNotFoundError: No module named 'self_py_fun.HW8Fun'

## Q2. A real-world data anlaysis using `Pandas` and `Scipy` (10')

<font size='4'>

- Back to the `PTSD dataset.xlsx`, let's import the dataset and name it `ptsd_df`. (no point since everyone has done it a couple of times before.)

In [1]:
# Write your own code


### Q2.1. Univariate comparison (3')

<font size='4'> 
    
- Suppose that we would like to examine the utility/effect of an intervention program for patients with PTSD.
- We measure PCL5 scores at completion (`pcl5week_score.completion`) and PCL5 score at 3-month follow-up (`pcl5month_score.3_month_follow_up`). Let's assume the first score is pre-intervention and the second score is post-intervention.
- Report the summary statistics for each variable including mean, std, median, Q1, and Q3.
- Note that each patient will receive such two PCL5 scores. Use a appropriate statistical test to perform the univariate comparison. Report the outputing statistic and p-value.
- Before you run the statistic test, determine the data type and check the missingness of two columns. In particular, report the number of NA values for each variable.

In [2]:
# Write your own code


### Q2.2. Multiple Linear Regression (7')

<font size='4'>

- Select columns specified in the following code chunk and create a subset dataset named `ptsd_sub_df`.
- Fit a linear regression to examine the association between `caps_intake` (continuous outcome) and the remaining covariates (as predictors) using `ptsd_sub_df`.
    - Note that all covariates ending with `_code` are categorical variables.
- Use the instruction here to write the formula for linear regression in Python.
    - https://www.statsmodels.org/stable/example_formulas.html
- Report the output page including R2, adjusted R2, and parameter estimates, SE, 95% confidence intervals, and p-values.
- Provide a brief interpretation for all significant predictors (p<0.05) excluding the intercept.
- Relevant label information includes:
    - `employment_code`: 1: Employed, 2: Unemployed, 3: Retired, 4: Disabled/Unable to work, 5: Student, 6: Other.
    - `rank_code`: 1. Enlisted, 2: Officer, 3: Other

In [89]:
# The following column names are used for linear regression.
# Do not delete.
relevant_col_names = ['caps_intake', 'age_iop', 'gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                      'education_code', 'employment_code',
                      'rank_code', 'branch_code', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code']

In [3]:
# Write your own code


In [88]:
# Write your interpretations below:
